# NLP 作業3

## 安裝需要的套件

In [1]:
#!pip install datasets
#!pip install evaluate
#!pip install seqeval

In [2]:
import datasets
import evaluate
import seqeval

print(datasets.__version__)
print(evaluate.__version__)

/home/hjc/venv/.venvPy01/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3.5.0
0.4.3


In [3]:
!pip list | grep seqeval

seqeval                   1.2.2


In [4]:
import torch

# 測試現在這個 Colab 環境是否已經使用 GPU
# 否則等下可能會需要重新啟動 Colab 環境
torch.cuda.is_available() # 結果需要顯示為 True

True

## Start

In [5]:
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import EvalPrediction

In [6]:
MODEL_NAME = "bert-base-uncased"
DATA_NAME = "ncbi_disease"
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 20
NUM_TRAIN_EPOCHS = 3
LEARNING_RATE = 2e-5

In [7]:
# 載入資料集
dataset = datasets.load_dataset(DATA_NAME, trust_remote_code=True)

In [8]:
# 檢查每個 split 的數量

for data_type in ["train", "validation", "test"]:
    print(f"{data_type}: {len(dataset[data_type])} samples")

train: 5433 samples
validation: 924 samples
test: 941 samples


In [9]:
# Show NER tag names
label_names = dataset["train"].features["ner_tags"].feature.names
print(label_names)

['O', 'B-Disease', 'I-Disease']


In [10]:
# 觀察資料
first_example = dataset["train"][1]
print(type(first_example))

for k, v in first_example.items():
    print(f"{k}: {v}")

<class 'dict'>
id: 1
tokens: ['The', 'adenomatous', 'polyposis', 'coli', '(', 'APC', ')', 'tumour', '-', 'suppressor', 'protein', 'controls', 'the', 'Wnt', 'signalling', 'pathway', 'by', 'forming', 'a', 'complex', 'with', 'glycogen', 'synthase', 'kinase', '3beta', '(', 'GSK', '-', '3beta', ')', ',', 'axin', '/', 'conductin', 'and', 'betacatenin', '.']
ner_tags: [0, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [11]:
# TODO1: 建立 tokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)# Write your code here


In [12]:
tokenized_inputs = tokenizer(
        first_example["tokens"],
        truncation=True,
        is_split_into_words=True,
)
print("Tokenized 後的結果：")
print(tokenized_inputs)
print("Tokenized 後的 word_ids：")
print(tokenized_inputs.word_ids())

print("原始資料的 labels：")
print(first_example["ner_tags"])
print("我們下一步需要轉換 labels 為：")
print("[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, -100]")

Tokenized 後的結果：
{'input_ids': [101, 1996, 16298, 9626, 24826, 2015, 26572, 6873, 6190, 27441, 1006, 9706, 2278, 1007, 10722, 20360, 1011, 16081, 2953, 5250, 7711, 1996, 1059, 3372, 21919, 12732, 2011, 5716, 1037, 3375, 2007, 1043, 2135, 3597, 6914, 24203, 11022, 21903, 1017, 20915, 2050, 1006, 28177, 2243, 1011, 1017, 20915, 2050, 1007, 1010, 22260, 2378, 1013, 6204, 2378, 1998, 8247, 16280, 11483, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Tokenized 後的 word_ids：
[None, 0, 1, 1, 1, 1, 2, 2, 2, 3, 4, 5, 5, 6, 7, 7, 8, 9, 9, 10, 11, 12, 13, 13, 14, 15, 16, 17, 18, 19, 20, 21, 21, 21, 21, 22, 22, 23, 24, 24, 24, 25, 26, 

In [13]:
# TODO2: labels 處理
# 我們可以發現資料集的 labels (ner_tags) 是以 word 為單位
# 但是我們要使用的 tokenizer (如 BERT 的 tokenizer) 會把 word 切成 subwords
# 因此要針對 labels 的部分進行處理

def tokenize_and_align_labels(example):
    original_labels = example["ner_tags"]

    tokenized_inputs = tokenizer(
        example["tokens"],
        truncation=True,
        is_split_into_words=True,
    )
    word_ids = tokenized_inputs.word_ids()
    labels = []
    current_word_idx = None
    for word_idx in word_ids:
        # Write your code here
        # Hints:
        # (1) [CLS] or [SEP] 設為 -100
        # (2) 由左至右給予新的 labels，
        # 因此需要 current_word_idx
        # 來幫助我們觀察下個 token 是否進到新的 word，還是是 上一個 word 的 sub-word
        if word_idx is None:
            labels.append(-100)
        elif word_idx != current_word_idx:
            labels.append(original_labels[word_idx])
        else:
            labels.append(-100)
    
    current_word_idx = word_idx
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [14]:
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=False,
    remove_columns=dataset["train"].column_names,
)

In [15]:
tokenized_datasets["train"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 5433
})

In [16]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [17]:
fake_batch = [tokenized_datasets["train"][i] for i in range(2)]
batch = data_collator(fake_batch)
batch["labels"]

tensor([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            1,    1,    1,    1,    2,    2,    2,    2,    2,    2,    0,    0,
            0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100],
        [-100,    0,    1,    1,    1,    1,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         -100]])

In [18]:
# TODO3: 建立 id -> label (`id2label`) 以及 label -> id (`label2id`) 的轉換
# `id2label` 和 `label2id` 都是 Python dict，且 key 跟 value 都是 int

id2label = {idx: label for idx, label in enumerate(label_names)} # Write your code here
label2id = {label: idx for idx, label in enumerate(label_names)} # Write your code here

print(id2label)
print(label2id)

{0: 'O', 1: 'B-Disease', 2: 'I-Disease'}
{'O': 0, 'B-Disease': 1, 'I-Disease': 2}


In [19]:
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# 檢查模型是否確實被設定為3種類別輸出
model.config.num_labels

3

In [21]:
# TODO4: 設定 TrainingArguments

training_args = TrainingArguments(
    output_dir="./bertNER",
    overwrite_output_dir=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    warmup_ratio=0.1,
    report_to='tensorboard',
    push_to_hub=False,
)

In [22]:
print("↳ max label in dataset :", max(dataset["train"].features["ner_tags"].feature.names))
print("↳ len(label_names)     :", len(label_names))
print("↳ model.config.num_labels :", model.config.num_labels)
print("↳ id2label keys range  :", sorted(id2label.keys()))


↳ max label in dataset : O
↳ len(label_names)     : 3
↳ model.config.num_labels : 3
↳ id2label keys range  : [0, 1, 2]


In [ ]:
# TODO5: 完成 compute_metrics
import numpy as np

metric = evaluate.load("seqeval")

def compute_metrics(eval_pred: EvalPrediction):
    predictions, labels = eval_pred.predictions, eval_pred.label_ids
    # Write your code here
    pred_ids = np.argmax(predictions, axis=2) # (batch, seq_len, num_labels)變成(batch, seq_len)
    true_pred, true_label = [], []
    for pred_seq, label_seq in zip(pred_ids, labels):
        sent_preds, sent_labels = [], []
        for p_id, l_id in zip(pred_seq, label_seq):
            if l_id == -100:
                continue
            sent_labels.append(id2label[int(l_id)])
            sent_preds.append(id2label[int(p_id)])
        true_label.append(sent_labels)
        true_pred.append(sent_preds)
    #ChatGPT fix this bug ▼
    metrics = metric.compute(predictions = true_pred, references = true_label) #seqeval 計算整體指標，這邊要注意是使用references當作變數名稱，按照Hugging Face 格式
    return { # Write your code here
        "precision": metrics["overall_precision"],
        "recall": metrics["overall_recall"],
        "f1": metrics["overall_f1"],
        "accuracy": metrics["overall_accuracy"],
    } 

In [24]:
# TODO6: set up trainer

trainer = Trainer(
    # Write your code here
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_11438/4136483146.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### 相關官方文件連結
- [TokenClassifierOutput](https://huggingface.co/docs/transformers/en/main_classes/output#transformers.modeling_outputs.TokenClassifierOutput)
- [EvalPrediction](https://huggingface.co/docs/transformers/internal/trainer_utils#transformers.EvalPrediction)

In [25]:
# compute_metrics 模擬測試區

eval_dataloader = trainer.get_eval_dataloader()
batch = next(iter(eval_dataloader))
with torch.no_grad():
    outputs = model(**{k: v.to(model.device) for k, v in batch.items()})
print(f"AutoModelForTokenClassification 輸出的型態為: {type(outputs)}")
print(f"Logits shape: {outputs.logits.shape}")
print(f"Labels shape: {batch['labels'].shape}")
print(f"Loss: {outputs.loss.item()}")
print(f"Type of `outputs.loss`: {type(outputs.loss)}")
print(f"Type of `outputs.loss.item()`: {type(outputs.loss.item())}")
print()

# 取得 logits 和 labels
logits = outputs.logits.cpu().numpy()
labels = batch["labels"].cpu().numpy()

# 建立 EvalPrediction 模擬 compute_metrics 呼叫
mock_eval = EvalPrediction(
    predictions=logits,
    label_ids=labels,
)
print(f"Trainer 在輸進去 compute_metrics 前的型態為: {type(mock_eval)}")

# 呼叫你自己寫的 metrics function
metrics = compute_metrics(mock_eval)
for k, v in metrics.items():
    print(f"{k}: {v}")

AutoModelForTokenClassification 輸出的型態為: <class 'transformers.modeling_outputs.TokenClassifierOutput'>
Logits shape: torch.Size([32, 59, 3])
Labels shape: torch.Size([32, 59])
Loss: 1.1592801809310913
Type of `outputs.loss`: <class 'torch.Tensor'>
Type of `outputs.loss.item()`: <class 'float'>

Trainer 在輸進去 compute_metrics 前的型態為: <class 'transformers.trainer_utils.EvalPrediction'>
precision: 0.01818181818181818
recall: 0.08108108108108109
f1: 0.029702970297029698
accuracy: 0.1509433962264151


In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.086400,0.071946,0.794100,0.850000,0.821099,0.978290
2,0.039700,0.069412,0.795931,0.875000,0.833594,0.981143
3,0.016800,0.075871,0.822808,0.882895,0.851793,0.982127


TrainOutput(global_step=510, training_loss=0.09135093817523882, metrics={'train_runtime': 82.1828, 'train_samples_per_second': 198.326, 'train_steps_per_second': 6.206, 'total_flos': 577037255854140.0, 'train_loss': 0.09135093817523882, 'epoch': 3.0})

In [28]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.08857279270887375,
 'eval_precision': 0.8365514144589133,
 'eval_recall': 0.881267738883633,
 'eval_f1': 0.8583275742916379,
 'eval_accuracy': 0.9748477543770616,
 'eval_runtime': 1.1154,
 'eval_samples_per_second': 843.625,
 'eval_steps_per_second': 26.896,
 'epoch': 3.0}

In [ ]:
#這次幾乎無使用GPT，中間有修BUG有使用ChatGPT-4o已標注